In [ ]:
import pandas as pd
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import re
from nltk.stem import WordNetLemmatizer
url = 'https://raw.githubusercontent.com/SyedT1/NLP/main/csv_files/IMDB%20Dataset.csv'
df = pd.read_csv(url)
df.drop(columns=df.columns[2:4], inplace=True)

In [ ]:
reviews = df['review']
df['sentiment'] = df['sentiment'].replace(['positive'], 1)
df['sentiment'] = df['sentiment'].replace(['negative'],0)
sentiments = df['sentiment']

In [ ]:
print(df.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...         1
1  A wonderful little production. <br /><br />The...         1
2  I thought this was a wonderful way to spend ti...         1
3  Basically there's a family where a little boy ...         0
4  Petter Mattei's "Love in the Time of Money" is...         1


In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import numpy as np
y = np.int32(df.sentiment.astype('category').cat.codes.to_numpy())
num_classes = np.unique(y).shape[0]
print(len(y))

16502


In [ ]:
stemmer = WordNetLemmatizer()
def custom_standardization(text):
    text = re.sub('<br />', ' ', str(text))
    text = re.sub(r'\W', ' ', str(text))
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    # substituting multiple spaces with single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    # removing prefixed 'b'
    text = re.sub(r'^b\s+', '', text)
    # converting to Lowercase
    text = text.lower()
    text = text.split()
    text = [stemmer.lemmatize(word) for word in text]
    text = ' '.join(text)
    return text
    pass
df['review'] = df.review.apply(custom_standardization)

In [ ]:
max_features = 10000
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
results = Counter()
df['review'].str.split().apply(results.update)
vocabulary = [key[0] for key in results.most_common(max_features)]
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(vocabulary)

In [ ]:

X = tokenizer.texts_to_sequences(df['review'].values)
X = pad_sequences(X)


# split and enzoy :/
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
results = Counter()
df['review'].str.split().apply(results.update)
vocabulary = [key[0] for key in results.most_common(max_features)]

10000


In [ ]:
output_dim = 16
max_input_length = X.shape[1]


In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=max_features, output_dim=output_dim, input_length=max_input_length))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(1e-4), metrics=["accuracy"])


model.summary()

history_1 = model.fit(X_train, y_train, batch_size=8,epochs=20, validation_data=(X_val, y_val))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1746, 16)          160000    
                                                                 
 dropout_2 (Dropout)         (None, 1746, 16)          0         
                                                                 
 global_average_pooling1d_2   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 16)                272       
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                                 
Total params: 160,544
Trainable params: 160,544
Non-trainable params: 0
________________________________________________

In [ ]:
probabilities = model.predict(X_test)
pred = np.argmax(probabilities, axis=1)

print(" ")
print("Results")

accuracy = accuracy_score(y_test, pred)

print('Shothik koddur: {:.6f}'.format(accuracy))
print(" ")
print(classification_report(y_test, pred))


NameError: ignored